In [1]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.chat_models import AzureChatOpenAI, ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from IPython.display import display, HTML
from langchain.chat_models import AzureChatOpenAI, ChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings
from openai import OpenAI, AzureOpenAI, AsyncAzureOpenAI
import os

In [2]:
llm = AzureChatOpenAI(
                azure_endpoint=os.getenv('OpenAiEndPoint'),
                api_version=os.getenv('OpenAiVersion', "2023-05-15"),
                azure_deployment=os.getenv('OpenAiChat'),
                temperature=0,
                api_key=os.getenv('OpenAiKey'),
                openai_api_type="azure",
                max_tokens=2000)
embeddings = AzureOpenAIEmbeddings(azure_endpoint=os.getenv('OpenAiEndPoint'), 
                                   azure_deployment=os.getenv('OpenAiEmbedding'), api_key=os.getenv('OpenAiKey'), openai_api_type="azure")

C:\Users\astalati\AppData\Roaming\Python\Python311\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.azure_openai.AzureChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(


In [5]:
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.schema.document import Document
from langchain.chains.mapreduce import MapReduceChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import ReduceDocumentsChain, MapReduceDocumentsChain

In [4]:
# Define prompt
prompt_template = """\n\nHuman:  Consider this text:
<text>
{text}
</text>
Please create a concise summary in narative format.

Assistiant:  Here is the concise summary:"""
prompt = PromptTemplate.from_template(prompt_template)

# Define LLM chain
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Define StuffDocumentsChain
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="text")

#Note that although langchain often stores douments in small chunks for the 
#convience of models with smaller context windows, this "stuff it" method will
#combind all those chunks into a single prompt call.

#wrapping in a python function to make it easy to use in other scripts.
def stuff_it_summary(doc):
    if type(doc) == str:
        docs = [Document(page_content=doc)]
    return stuff_chain.run(docs)

In [6]:
# Map
map_template = """\n\nHuman: The following is a set of documents
<documnets>
{docs}
</documents>
Based on this list of docs, please identify the main themes.

Assistant:  Here are the main themes:"""
map_prompt = PromptTemplate.from_template(map_template)
map_chain = LLMChain(llm=llm, prompt=map_prompt)

# Reduce
reduce_template = """\n\nHuman: The following is set of summaries:
<summaries>
{doc_summaries}
</summaries>
Please take these and distill them into a final, consolidated summary of the main themes in narative format. 

Assistant:  Here are the main themes:"""
reduce_prompt = PromptTemplate.from_template(reduce_template)
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

# Takes a list of documents, combines them into a single string, and passes this to an LLMChain
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="doc_summaries"
)

# Combines and iteravely reduces the mapped documents
reduce_documents_chain = ReduceDocumentsChain(
    # This is final chain that is called.
    combine_documents_chain=combine_documents_chain,
    # If documents exceed context for `StuffDocumentsChain`
    collapse_documents_chain=combine_documents_chain,
    # The maximum number of tokens to group documents into.
    token_max=4000,
)

# Combining documents by mapping a chain over them, then combining results
map_reduce_chain = MapReduceDocumentsChain(
    # Map chain
    llm_chain=map_chain,
    # Reduce chain
    reduce_documents_chain=reduce_documents_chain,
    # The variable name in the llm_chain to put the documents in
    document_variable_name="docs",
    # Return the results of the map steps in the output
    return_intermediate_steps=False,
)

#wrapping in a python function to make it easy to use in other scripts.
def map_reduce_summary(doc, DEBUG=False):
    if type(doc) == str:
        #use the LangChain built in text splitter to split our text
        from langchain.text_splitter import RecursiveCharacterTextSplitter
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size = 5000,
            chunk_overlap  = 200,
            length_function = len,
            add_start_index = True,
        )
        split_docs = text_splitter.create_documents([doc])
        if DEBUG: print("Text was split into %s docs"%len(split_docs))
    return map_reduce_chain.run(split_docs)

In [ ]:
text_to_open_short = './Data/Pickle/hills.pkl'  #2-3 page story, Hills like White Elephants
text_to_open_mid = './Data/Pickle/algernon.pkl'  #short story, Flowers for Algernon
text_to_open_long = './Data/Pickle/frankenstien.pkl' #short novel, Frankenstine
text_to_open_short_factual = './Data/Pickle/elvis.pkl'  #longest wikipedia article, Elvis.

from langchain.schema.document import Document

with open(text_to_open_short, 'rb') as file:
    #note that here, we're loading a single text, but the examples below require each text to be in a list.
    doc = pickle.load(file)